# <B> 1.데이터정제

## <b> (1) 결측치

### <B> isnull()의 기본 사용법

In [1]:
import pandas as pd 
# 예제 데이터프레임 생성 

data = {'Name': ['Alice', 'Bob', 'Charlie'], # 이름 
         'Age': [25, None, 30], # 나이 (Bob의 나이가 비어 있음)
         'Score': [90, 85, None]} # 점수 (Charlie's 점수가 비어 있음) 
df = pd.DataFrame(data) 

# 결측치 여부 확인 
print("결측치 여부 확인:")
print(df.isnull())

결측치 여부 확인:
    Name    Age  Score
0  False  False  False
1  False   True  False
2  False  False   True


In [2]:
df.isnull()

,Name,Age,Score
0,False,False,False
1,False,True,False
2,False,False,True


### <B> isnull().sum()으로 결측치 개수 확인하기

In [3]:
# 컬럼별 결측치 개수 확인
print("\n컬럼별 결측치 개수:")
print(df.isnull().sum())


컬럼별 결측치 개수:
Name     0
Age      1
Score    1
dtype: int64


In [4]:
# 결측치가 있는 행 삭제
df_cleaned = df.dropna()
print("\n결측치 삭제 후 데이터:")
print(df_cleaned)


결측치 삭제 후 데이터:
    Name   Age  Score
0  Alice  25.0   90.0


In [5]:
# Age 열의 평균값으로 결측치 대체
mean_age = df['Age'].mean()  # 평균값 계산
df['Age'] = df['Age'].fillna(mean_age)
print("\nAge 열 평균값으로 결측치 대체:")
print(df)


Age 열 평균값으로 결측치 대체:
      Name   Age  Score
0    Alice  25.0   90.0
1      Bob  27.5   85.0
2  Charlie  30.0    NaN


In [6]:
# Score 열의 결측치를 0으로 대체
df['Score'] = df['Score'].fillna(0)
print("\nScore 열 0으로 결측치 대체:")
print(df)


Score 열 0으로 결측치 대체:
      Name   Age  Score
0    Alice  25.0   90.0
1      Bob  27.5   85.0
2  Charlie  30.0    0.0


In [7]:
import pandas as pd

# 데이터 생성
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, None, 30],
        'Score': [90, 85, None]}
df = pd.DataFrame(data)

# 결측치 확인
print("결측치 확인:")
print(df.isnull().sum())

# 결측치 처리
# 1. Age 열의 결측치를 평균값으로 대체
mean_age = df['Age'].mean()
df['Age'] = df['Age'].fillna(mean_age)

# 2. Score 열의 결측치를 0으로 대체
df['Score'] = df['Score'].fillna(0)

print("\n결측치 처리 후 데이터:")
print(df)

결측치 확인:
Name     0
Age      1
Score    1
dtype: int64

결측치 처리 후 데이터:
      Name   Age  Score
0    Alice  25.0   90.0
1      Bob  27.5   85.0
2  Charlie  30.0    0.0


## <b> (2) 이상치

### <b> 기본적인 이상치 확인 방법

In [8]:
import pandas as pd
# 예제 데이터프레임 생성
data = {'Height': [150, 160, 170, 180, 350]}  # 300은 이상치로 가정
df = pd.DataFrame(data)
# 데이터 출력
print("데이터:\n", df)
# 최대값과 최소값 확인
print("\nHeight의 최대값:", df['Height'].max())
print("Height의 최소값:", df['Height'].min())

데이터:
    Height
0     150
1     160
2     170
3     180
4     350

Height의 최대값: 350
Height의 최소값: 150


### <b> IQR(Interquartile Range) 방법

In [9]:
# IQR을 이용한 이상치 확인
Q1 = df['Height'].quantile(0.25)  # 1사분위 값
Q3 = df['Height'].quantile(0.75)  # 3사분위 값
IQR = Q3 - Q1  # IQR 계산
# 이상치 기준 계산
lower_bound = Q1 - 1.5 * IQR  # 하한
upper_bound = Q3 + 1.5 * IQR  # 상한
# 이상치 확인
outliers = df[(df['Height'] < lower_bound) | (df['Height'] > upper_bound)]
print("\nIQR 기준으로 찾은 이상치:\n", outliers)


IQR 기준으로 찾은 이상치:
    Height
4     350


### <b> Z-스코어를 활용한 이상치 확인

In [10]:
from scipy.stats import zscore
# Z-스코어 계산
df['Z_Score'] = zscore(df['Height'])  # Z-스코어 계산
print("\nZ-스코어:\n", df)
# Z-스코어 기준으로 이상치 탐지
outliers = df[df['Z_Score'].abs() > 3]
print("\nZ-스코어 기준으로 찾은 이상치:\n", outliers)


Z-스코어:
    Height   Z_Score
0     150 -0.696373
1     160 -0.562455
2     170 -0.428537
3     180 -0.294619
4     350  1.981985

Z-스코어 기준으로 찾은 이상치:
 Empty DataFrame
Columns: [Height, Z_Score]
Index: []


### <b> IQR기준 이상치 제거

In [11]:
# IQR 기준으로 이상치 제거
df_cleaned = df[(df['Height'] >= lower_bound) & (df['Height'] <= upper_bound)]
print("\n이상치 제거 후 데이터:\n", df_cleaned)


이상치 제거 후 데이터:
    Height   Z_Score
0     150 -0.696373
1     160 -0.562455
2     170 -0.428537
3     180 -0.294619


### <b> 이상치를 특정 값으로 대체

#### <b> IQR(Interquartile Range) 방법

In [12]:
# 이상치를 중앙값으로 대체
median_value = df[(df['Height'] >= lower_bound) & (df['Height'] <= upper_bound)]['Height'].median()
df['Height'] = df['Height'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)
print("\n이상치를 중앙값으로 대체한 데이터:\n", df)


이상치를 중앙값으로 대체한 데이터:
    Height   Z_Score
0   150.0 -0.696373
1   160.0 -0.562455
2   170.0 -0.428537
3   180.0 -0.294619
4   165.0  1.981985


#### <b> 이상치 제거 전체코드

In [13]:
import pandas as pd
from scipy.stats import zscore

# 데이터 생성
data = {'Height': [150, 160, 170, 180, 350]}
df = pd.DataFrame(data)
# IQR 방법
Q1 = df['Height'].quantile(0.25)
Q3 = df['Height'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Z-스코어 계산
df['Z_Score'] = zscore(df['Height'])

# 이상치 제거
df_cleaned = df[(df['Height'] >= lower_bound) & (df['Height'] <= upper_bound)]

# 이상치를 중앙값으로 대체
median_value = df_cleaned['Height'].median()
df['Height'] = df['Height'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)

print("최종 데이터:\n", df)

최종 데이터:
    Height   Z_Score
0   150.0 -0.696373
1   160.0 -0.562455
2   170.0 -0.428537
3   180.0 -0.294619
4   165.0  1.981985


## <b> (3)수치형 데이터 전처리

### <b> 예제 : 나이 데이터를 구간화하기

In [14]:
import pandas as pd

# 데이터 생성
data = {'Age': [15, 22, 35, 50, 72]}
df = pd.DataFrame(data)

# 나이를 구간화 (10대, 20대, 30대, ...)
bins = [0, 20, 40, 60, 80]  # 구간 경계 설정
labels = ['10대', '20대', '30대', '40대 이상']  # 각 구간의 이름
df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels)

# 결과 출력
print(df)


   Age Age_Group
0   15       10대
1   22       20대
2   35       20대
3   50       30대
4   72    40대 이상


### <b> 구간화를 자동으로 수행하기

In [15]:
# 나이를 자동으로 3개의 구간으로 나누기
df['Age_Quantile'] = pd.qcut(df['Age'], q=3, labels=['하위', '중위', '상위'])
# 결과 출력
print(df)

   Age Age_Group Age_Quantile
0   15       10대           하위
1   22       20대           하위
2   35       20대           중위
3   50       30대           상위
4   72    40대 이상           상위


### <b> 예제: MinMaxScaler를 사용한 정규화

In [16]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# 데이터 생성
data = {'Height': [150, 160, 170, 180, 190]}
df = pd.DataFrame(data)

# MinMaxScaler 적용
scaler = MinMaxScaler()
df['Height_Normalized'] = scaler.fit_transform(df[['Height']])

# 결과 출력
print(df)

   Height  Height_Normalized
0     150               0.00
1     160               0.25
2     170               0.50
3     180               0.75
4     190               1.00


### <b> 예제: StandardScaler를 사용한 표준화

In [17]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 데이터 생성
data = {'Weight': [50, 60, 70, 80, 90]}
df = pd.DataFrame(data)

# StandardScaler 적용
scaler = StandardScaler()
df['Weight_Standardized'] = scaler.fit_transform(df[['Weight']])

# 결과 출력
print(df)

   Weight  Weight_Standardized
0      50            -1.414214
1      60            -0.707107
2      70             0.000000
3      80             0.707107
4      90             1.414214


### <b>  스케일링 전체 코드

In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 데이터 생성
data = {'Age': [15, 22, 35, 50, 72], 'Weight': [50, 60, 70, 80, 90]}
df = pd.DataFrame(data)

# 구간화
bins = [0, 20, 40, 60, 80]
labels = ['10대', '20대', '30대', '40대 이상']
df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels)

# 정규화
minmax_scaler = MinMaxScaler()
df['Weight_Normalized'] = minmax_scaler.fit_transform(df[['Weight']])

# 표준화
std_scaler = StandardScaler()
df['Weight_Standardized'] = std_scaler.fit_transform(df[['Weight']])
print(df)

   Age  Weight Age_Group  Weight_Normalized  Weight_Standardized
0   15      50       10대               0.00            -1.414214
1   22      60       20대               0.25            -0.707107
2   35      70       20대               0.50             0.000000
3   50      80       30대               0.75             0.707107
4   72      90    40대 이상               1.00             1.414214


## <b> (4)범주형 데이터 전처리

### <b> 예제코드 : 레이블 인코딩

In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder  # 레이블 인코딩을 위한 라이브러리

# 데이터 생성
data = {'City': ['서울', '부산', '대구', '서울', '부산']}
df = pd.DataFrame(data)

# 레이블 인코딩
encoder = LabelEncoder()  # 레이블 인코더 생성
df['City_Encoded'] = encoder.fit_transform(df['City'])  # 범주형 데이터를 숫자로 변환
# 결과 출력
print(df)

  City  City_Encoded
0   서울             2
1   부산             1
2   대구             0
3   서울             2
4   부산             1


### <b> 예제코드 : 원핫 인코딩

In [20]:
import pandas as pd

# 데이터 생성
data = {'City': ['서울', '부산', '대구', '서울', '부산']}
df = pd.DataFrame(data)
# 원핫 인코딩
df_encoded = pd.get_dummies(df, columns=['City'])  # City 열을 원핫 인코딩

# 결과 출력
print(df_encoded)

   City_대구  City_부산  City_서울
0    False    False     True
1    False     True    False
2     True    False    False
3    False    False     True
4    False     True    False


In [21]:
import pandas as pd
heart_data = pd.read_csv("datasets/heart.csv")
heart_data.columns
print(heart_data.head(4))

   age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
0   63    1   3     145   233    1        0       150     0      2.3    0   
1   37    1   2     130   250    0        1       187     0      3.5    0   
2   41    0   1     130   204    0        0       172     0      1.4    2   
3   56    1   1     120   236    0        1       178     0      0.8    2   

   caa  thall  output  
0    0      1       1  
1    0      2       1  
2    0      2       1  
3    0      2       1  


## <b> (5)기존 데이터 삭제

### <b> 기존 데이터 삭제 : 특정 행(Row) 삭제


In [16]:
import pandas as pd

# 데이터 생성
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40],
        'Score': [90, 85, 75, 60]}
df = pd.DataFrame(data)
df

,Name,Age,Score
0,Alice,25,90
1,Bob,30,85
2,Charlie,35,75
3,David,40,60


In [14]:
# 특정 행 삭제 (예: 인덱스 2번 삭제)
df_dropped = df.drop(index=2)
df_dropped 

,Name,Age,Score
0,Alice,25,90
1,Bob,30,85
3,David,40,60


In [18]:
df_dropped_col = df.drop(columns = ['Score', 'Age'])
df_dropped_col

,Name
0,Alice
1,Bob
2,Charlie
3,David


## <b> 기존 데이터 삭제 : 특정 열(column) 삭제


In [23]:
# 특정 열 삭제 (예: 'Score' 열 삭제)
df_dropped = df.drop(columns=['Score'])

특정 열 삭제 후 데이터:
      Name  Age
0    Alice   25
1      Bob   30
2  Charlie   35
3    David   40


### <b> 기존 데이터 삭제 : 결측치가 많은 행(Row) 삭제


In [3]:
# 결측치가 있는 데이터 생성
import pandas as pd
data_with_na = {'Name': ['Alice', 'Bob', 'Charlie', 'David'],
                'Age': [25, None, 35, 40],
                'Score': [90, 85, None, 60]}
df_with_na = pd.DataFrame(data_with_na)
df_with_na

,Name,Age,Score
0,Alice,25.0,90.0
1,Bob,NaN,85.0
2,Charlie,35.0,NaN
3,David,40.0,60.0


In [10]:
# 결측치가 포함된 행 삭제
df_cleaned = df_with_na.dropna( axis = 0)
df_cleaned


,Name,Age,Score
0,Alice,25.0,90.0
3,David,40.0,60.0


In [8]:
df_cleaned_col = df_with_na.dropna(axis = 1 )
df_cleaned_col

,Name
0,Alice
1,Bob
2,Charlie
3,David


### <b> 기존 데이터 삭제 : 중복 데이터 삭제



In [19]:
# 중복 데이터 생성
data_with_duplicates = {'Name': ['Alice', 'Bob', 'Alice', 'David'],
                        'Age': [25, 30, 25, 40],
                        'Score': [90, 85, 90, 60]}
df_with_duplicates = pd.DataFrame(data_with_duplicates)
df_with_duplicates

,Name,Age,Score
0,Alice,25,90
1,Bob,30,85
2,Alice,25,90
3,David,40,60


In [21]:
df_unique =df_with_duplicates.drop_duplicates()
df_unique

,Name,Age,Score
0,Alice,25,90
1,Bob,30,85
3,David,40,60


### <b> 기존 데이터 삭제 : 실습 예제


In [22]:
import pandas as pd

# 데이터 생성
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
        'Age': [25, None, 35, 40, 29],
        'Score': [90, 85, 75, 60, 85]}
df = pd.DataFrame(data)

# 1. 특정 행 삭제 (예: 인덱스 1번 삭제)
df = df.drop(index=1)

# 2. 특정 열 삭제 (예: 'Score' 열 삭제)
df = df.drop(columns=['Score'])

# 3. 결측치가 포함된 행 삭제
df = df.dropna()
print("정제된 데이터:")
print(df)

정제된 데이터:
      Name   Age
0    Alice  25.0
2  Charlie  35.0
3    David  40.0
4      Eve  29.0


## <b> (6)컬럼명 변경

In [23]:
import pandas as pd
# 데이터 로드
file_path = "datasets/heart.csv"
df = pd.read_csv(file_path)

In [24]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [29]:
df.columns

Index(['나이', '성별', '흉통유형', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

### 방법 1: rename() 메서드로 특정 컬럼명 변경

In [28]:
# 특정 컬럼명 변경
df.rename(columns={'age': '나이', 'sex': '성별', 'cp': '흉통유형'}, inplace=True)
df

,나이,성별,흉통유형,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


### 방법 2: columns 속성으로 전체 컬럼명 변경

In [30]:
# 전체 컬럼명 변경
df.columns = ['나이', '성별', '흉통유형', '안정시혈압', '콜레스테롤', '공복혈당', '심전도',
              '최대심박수', '운동유발협심증', 'ST우울증', '운동후ST경사', '주요혈관수',
              '결함유형', '심장질환여부']
# 변경 후 컬럼명 확인
print("변경된 컬럼명 (전체 변경):")
print(df.columns)

변경된 컬럼명 (전체 변경):
Index(['나이', '성별', '흉통유형', '안정시혈압', '콜레스테롤', '공복혈당', '심전도', '최대심박수', '운동유발협심증',
       'ST우울증', '운동후ST경사', '주요혈관수', '결함유형', '심장질환여부'],
      dtype='object')


In [31]:
# 변경 후 데이터 확인
print("변경된 데이터 상위 5개 행:")
df.head()

변경된 데이터 상위 5개 행:


,나이,성별,흉통유형,안정시혈압,콜레스테롤,공복혈당,심전도,최대심박수,운동유발협심증,ST우울증,운동후ST경사,주요혈관수,결함유형,심장질환여부
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## <b> (7) 데이터정렬

In [32]:
import pandas as pd

# 데이터 로드
file = "datasets/heart.csv"
df = pd.read_csv(file)
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


### 특정 열 기준으로 정렬하기

In [33]:
# 나이(age) 기준으로 오름차순 정렬
df_sorted_by_age = df.sort_values(by='age', ascending=True)  #descending, ascending
df_sorted_by_age

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
72,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
58,34,1,3,118,182,0,0,174,0,0.0,2,0,2,1
125,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
239,35,1,0,126,282,0,0,156,1,0.0,2,0,3,0
65,35,0,0,138,183,0,1,182,0,1.4,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1
151,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1
129,74,0,1,120,269,0,0,121,1,0.2,2,1,2,1
144,76,0,2,140,197,0,2,116,0,1.1,1,0,2,1


### 여러 열 기준으로 정렬하기

In [34]:
# 나이(age)와 최대심박수(thalachh) 기준으로 정렬
df_sorted_by_multiple = df.sort_values(by=['age', 'thalachh'], ascending=[True, False])
df_sorted_by_multiple.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
72,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
125,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
58,34,1,3,118,182,0,0,174,0,0.0,2,0,2,1
65,35,0,0,138,183,0,1,182,0,1.4,2,0,2,1
157,35,1,1,122,192,0,1,174,0,0.0,2,0,2,1


### 인덱스 기준으로 정렬하기

In [34]:
# 인덱스 기준으로 정렬
df_sorted_by_index = df.sort_index(ascending=False)

# 정렬된 데이터 확인
print("인덱스 기준 내림차순 정렬:")
df_sorted_by_index.head()

인덱스 기준 내림차순 정렬:


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0


### 열 이름 기준으로 정렬하기

In [35]:
print("열 이름 기준 오름차순 정렬 전:")
df.head()

열 이름 기준 오름차순 정렬 전:


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [35]:
# 열 이름 기준으로 오름차순 정렬
df_sorted_by_columns = df.sort_index(axis=1, ascending=True)
df_sorted_by_columns.head()

,age,caa,chol,cp,exng,fbs,oldpeak,output,restecg,sex,slp,thalachh,thall,trtbps
0,63,0,233,3,0,1,2.3,1,0,1,0,150,1,145
1,37,0,250,2,0,0,3.5,1,1,1,0,187,2,130
2,41,0,204,1,0,0,1.4,1,0,0,2,172,2,130
3,56,0,236,1,0,0,0.8,1,1,1,2,178,2,120
4,57,0,354,0,1,0,0.6,1,1,0,2,163,2,120


## <b> (8)데이터 전처리 함수

### <b>①수치형 데이터 파생 함수 (숫자 데이터 다루기) : (실습파일 : heart.csv)

In [5]:
import pandas as pd

# 예제 데이터프레임 생성
df = pd.read_csv("datasets/heart.csv")

# (1) 최대 심박수 대비 나이 비율
df['heart_age_ratio'] = (df['thalachh'] / df['age']).round(2)

# (2) 콜레스테롤 누적합
df['chol_cumsum'] = df['chol'].cumsum()

# (3) 나이 그룹화 (10년 단위)
df['age_group'] = (df['age'] // 10) * 10

# (4) 심박수 순위
df['thalach_rank'] = df['thalachh'].rank()

# (5) 콜레스테롤 변화량 (diff)
df['chol_diff'] = df['chol'].diff()

print(df)


     age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
0     63    1   3     145   233    1        0       150     0      2.3    0   
1     37    1   2     130   250    0        1       187     0      3.5    0   
2     41    0   1     130   204    0        0       172     0      1.4    2   
3     56    1   1     120   236    0        1       178     0      0.8    2   
4     57    0   0     120   354    0        1       163     1      0.6    2   
..   ...  ...  ..     ...   ...  ...      ...       ...   ...      ...  ...   
298   57    0   0     140   241    0        1       123     1      0.2    1   
299   45    1   3     110   264    0        1       132     0      1.2    1   
300   68    1   0     144   193    1        1       141     0      3.4    1   
301   57    1   0     130   131    0        1       115     1      1.2    1   
302   57    0   1     130   236    0        0       174     0      0.0    1   

     caa  thall  output  heart_age_ratio  chol_cums

### <b>②	문자형 데이터 파생 함수

In [6]:
import pandas as pd

# 예제 데이터프레임 생성
df = pd.DataFrame({
    'name': [' 김철수 ', '이영희', '박민수', '정수진', '이서준'],
    'email': ['kim@naver.com', 'lee@gmail.com', 'park@daum.net', 'jung@hanmail.net', 'seo@korea.kr'],
    'gender_eng': ['Male', 'Female', 'Male', 'Female', 'Male']
})

# (1) 문자열 길이 계산
df['name_length'] = df['name'].str.len()

# (2) 대문자/소문자 변환
df['gender_upper'] = df['gender_eng'].str.upper()
df['gender_lower'] = df['gender_eng'].str.lower()

# (3) 특정 문자 포함 여부 확인 (이름에 '수'가 포함됐는가?)
df['has_수'] = df['name'].str.contains('수')

# (4) 문자열 바꾸기 (영문 → 한글)
df['gender_kr'] = df['gender_eng'].str.replace('Male', '남성').str.replace('Female', '여성')

# (5) 문자열 자르기 (이름 첫 글자)
df['first_letter'] = df['name'].str.slice(0, 1)

# (6) 문자열 나누기: 이메일에서 도메인 추출
df['email_domain'] = df['email'].str.split('@').str[1]

# (7) 공백 제거
df['name_stripped'] = df['name'].str.strip()

# (8) 문자열 연결: 이름 + 도메인 합치기
df['name_email_concat'] = df['name_stripped'] + '_' + df['email_domain']

print(df[['name', 'name_length', 'name_stripped', 'first_letter', 'has_수',
          'gender_eng', 'gender_upper', 'gender_lower', 'gender_kr',
          'email', 'email_domain', 'name_email_concat']])



    name  name_length name_stripped first_letter  has_수 gender_eng  \
0   김철수             5           김철수                True       Male   
1    이영희            3           이영희            이  False     Female   
2    박민수            3           박민수            박   True       Male   
3    정수진            3           정수진            정   True     Female   
4    이서준            3           이서준            이  False       Male   

  gender_upper gender_lower gender_kr             email email_domain  \
0         MALE         male        남성     kim@naver.com    naver.com   
1       FEMALE       female        여성     lee@gmail.com    gmail.com   
2         MALE         male        남성     park@daum.net     daum.net   
3       FEMALE       female        여성  jung@hanmail.net  hanmail.net   
4         MALE         male        남성      seo@korea.kr     korea.kr   

  name_email_concat  
0     김철수_naver.com  
1     이영희_gmail.com  
2      박민수_daum.net  
3   정수진_hanmail.net  
4      이서준_korea.kr  


### <b>③	날짜형 데이터 파생 함수

In [7]:
import pandas as pd

# (1) 예제 데이터 생성
df = pd.DataFrame({
    '가입일': pd.to_datetime(['2022-01-15', '2023-03-10', '2023-12-01', '2024-05-03', '2024-10-20'])
})
# (2) 기준일 설정
df['today'] = pd.to_datetime('2025-05-05')

# (3) 날짜 파생변수 생성
df['가입연도'] = df['가입일'].dt.year              # 연도
df['가입월'] = df['가입일'].dt.month              # 월
df['가입일자'] = df['가입일'].dt.day              # 일
df['가입요일'] = df['가입일'].dt.day_name()       # 요일명
df['가입요일번호'] = df['가입일'].dt.weekday       # 0=월 ~ 6=일
df['가입_분기'] = df['가입일'].dt.quarter          # 1~4분기
df['가입_주차'] = df['가입일'].dt.isocalendar().week  # ISO 기준 주차
df['월_시작일인가'] = df['가입일'].dt.is_month_start # 월 첫날?
df['월_마지막일인가'] = df['가입일'].dt.is_month_end # 월 마지막일?
df['가입후_경과일수'] = (df['today'] - df['가입일']).dt.days  # 경과 일수

# (4) 주말 여부 판단
df['주말가입여부'] = df['가입요일번호'].isin([5, 6])  # 토/일 = True

print(df)


         가입일      today  가입연도  가입월  가입일자      가입요일  가입요일번호  가입_분기  가입_주차  \
0 2022-01-15 2025-05-05  2022    1    15  Saturday       5      1      2   
1 2023-03-10 2025-05-05  2023    3    10    Friday       4      1     10   
2 2023-12-01 2025-05-05  2023   12     1    Friday       4      4     48   
3 2024-05-03 2025-05-05  2024    5     3    Friday       4      2     18   
4 2024-10-20 2025-05-05  2024   10    20    Sunday       6      4     42   

   월_시작일인가  월_마지막일인가  가입후_경과일수  주말가입여부  
0    False     False      1206    True  
1    False     False       787   False  
2     True     False       521   False  
3    False     False       367   False  
4    False     False       197    True  


# <b>2. 데이터 구조 변경

### <b> Heart Data 불러오기 및 컬럼명 변경

In [37]:
import pandas as pd
# 데이터 로드
file_path = "datasets/heart.csv"
df = pd.read_csv(file_path)
# 컬럼명 변경
df.columns = ['나이', '성별', '흉통유형', '안정시혈압', '콜레스테롤', '공복혈당', '심전도',
              '최대심박수', '운동유발협심증', 'ST우울증', '운동후ST경사', '주요혈관수',
              '결함유형', '심장질환여부']
# 데이터 확인
print("변경된 데이터프레임 컬럼명:")
print(df.columns)
print("\n데이터 상위 5개 행:")
print(df.head())

변경된 데이터프레임 컬럼명:
Index(['나이', '성별', '흉통유형', '안정시혈압', '콜레스테롤', '공복혈당', '심전도', '최대심박수', '운동유발협심증',
       'ST우울증', '운동후ST경사', '주요혈관수', '결함유형', '심장질환여부'],
      dtype='object')

데이터 상위 5개 행:
   나이  성별  흉통유형  안정시혈압  콜레스테롤  공복혈당  심전도  최대심박수  운동유발협심증  ST우울증  운동후ST경사  \
0  63   1     3    145    233     1    0    150        0    2.3        0   
1  37   1     2    130    250     0    1    187        0    3.5        0   
2  41   0     1    130    204     0    0    172        0    1.4        2   
3  56   1     1    120    236     0    1    178        0    0.8        2   
4  57   0     0    120    354     0    1    163        1    0.6        2   

   주요혈관수  결함유형  심장질환여부  
0      0     1       1  
1      0     2       1  
2      0     2       1  
3      0     2       1  
4      0     2       1  


## <b> 성별기준 그룹화

In [38]:
# 성별을 기준으로 데이터 그룹화
grouped = df.groupby('성별')

# 그룹별 데이터 확인
print("그룹화된 데이터 (성별 기준):")
grouped.size()  # 각 그룹의 행 개수 출력

그룹화된 데이터 (성별 기준):


성별
0     96
1    207
dtype: int64

## <b> 성별기준 열의 평균 집계

In [39]:
# 성별별 평균 계산
grouped_mean = grouped.mean()

# 그룹화 후 평균값 출력
print("\n성별별 주요 열의 평균:")
print(grouped_mean)


성별별 주요 열의 평균:
           나이      흉통유형       안정시혈압       콜레스테롤     공복혈당       심전도  \
성별                                                                   
0   55.677083  1.041667  133.083333  261.302083  0.12500  0.572917   
1   53.758454  0.932367  130.946860  239.289855  0.15942  0.507246   

         최대심박수   운동유발협심증     ST우울증   운동후ST경사     주요혈관수      결함유형    심장질환여부  
성별                                                                          
0   151.125000  0.229167  0.876042  1.427083  0.552083  2.125000  0.750000  
1   148.961353  0.371981  1.115459  1.386473  0.811594  2.400966  0.449275  


## <b> 여러 집계 함수 적용하기

In [40]:
# 성별별 최대값과 최소값 계산
grouped_stats = grouped.agg({
    '나이': ['max', 'min'],  # 나이의 최대값과 최소값
    '콜레스테롤': ['mean', 'std']  # 콜레스테롤의 평균과 표준편차
})
# 결과 출력
print("\n성별별 나이와 콜레스테롤 통계:")
grouped_stats


성별별 나이와 콜레스테롤 통계:


나이           콜레스테롤           
   max min        mean        std
성별                               
0   76  34  261.302083  65.088946
1   77  29  239.289855  42.782392

## <b> 여러 집계 함수 적용하기

In [41]:
# 성별과 흉통유형을 기준으로 그룹화
grouped_multi = df.groupby(['성별', '흉통유형']).mean()
# 결과 출력
print("\n성별과 흉통유형별 주요 열의 평균:")
grouped_multi


성별과 흉통유형별 주요 열의 평균:


나이       안정시혈압       콜레스테롤      공복혈당       심전도       최대심박수  \
성별 흉통유형                                                                      
0  0     57.256410  138.589744  267.538462  0.128205  0.538462  145.282051   
   1     51.944444  128.055556  251.444444  0.111111  0.555556  162.833333   
   2     54.971429  127.885714  261.057143  0.114286  0.600000  151.800000   
   3     63.250000  147.500000  247.000000  0.250000  0.750000  149.500000   
1  0     55.105769  129.557692  243.605769  0.125000  0.451923  138.759615   
   1     51.031250  128.593750  241.031250  0.093750  0.656250  162.187500   
   2     52.538462  132.057692  231.134615  0.250000  0.596154  158.173077   
   3     54.315789  139.473684  235.052632  0.210526  0.315789  157.315789   

          운동유발협심증     ST우울증   운동후ST경사     주요혈관수      결함유형    심장질환여부  
성별 흉통유형                                                              
0  0     0.461538  1.356410  1.179487  0.820513  2.282051  0.461538  
   1     0.111111  0.461111  1.666667  0.555556  2.000000  0.888889  
   2     0.057143  0.474286  1.571429  0.257143  2.028571  0.971429  
   3     0.000000  1.575000  1.500000  0.500000  2.000000  1.000000  
1  0     0.596154  1.393269  1.288462  1.019231  2.509615  0.201923  
   1     0.062500  0.234375  1.687500  0.343750  2.218750  0.781250  
   2     0.173077  1.015385  1.461538  0.807692  2.326923  0.673077  
   3     0.210526  1.352632  1.210526  0.473684  2.315789  0.631579

## <b> 피벗테이블

In [36]:
import pandas as pd
# 데이터 로드
file_path = "datasets/heart.csv"
df = pd.read_csv(file_path)
# 컬럼명 변경
df.columns = ['나이', '성별', '흉통유형', '안정시혈압', '콜레스테롤', '공복혈당', '심전도',
              '최대심박수', '운동유발협심증', 'ST우울증', '운동후ST경사', '주요혈관수',
              '결함유형', '심장질환여부']

In [38]:
df.head()

,나이,성별,흉통유형,안정시혈압,콜레스테롤,공복혈당,심전도,최대심박수,운동유발협심증,ST우울증,운동후ST경사,주요혈관수,결함유형,심장질환여부
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [39]:
# 피벗테이블 생성
pivot = df.pivot_table(index='성별', columns='흉통유형', values='최대심박수', aggfunc='mean')
# 결과 출력
print("성별과 흉통유형별 평균 최대심박수:")
pivot

성별과 흉통유형별 평균 최대심박수:


흉통유형,0,1,2,3
성별,,,,
0,145.282051,162.833333,151.800000,149.500000
1,138.759615,162.187500,158.173077,157.315789


In [40]:
# 피벗테이블 생성
신성진 = df.pivot_table(index='성별', columns='흉통유형', values='최대심박수', aggfunc=['mean', 'max'])

# 결과 출력
print("\n성별과 흉통유형별 평균 및 최대값:")
신성진


성별과 흉통유형별 평균 및 최대값:


mean                                      max               
흉통유형           0           1           2           3    0    1    2    3
성별                                                                      
0     145.282051  162.833333  151.800000  149.500000  182  192  179  171
1     138.759615  162.187500  158.173077  157.315789  186  202  194  190

In [45]:
# 피벗테이블 생성
pivot_multi_index = df.pivot_table(index=['성별', '흉통유형'], values='최대심박수', aggfunc='mean')

# 결과 출력
print("\n성별과 흉통유형을 인덱스로 한 피벗테이블:")
pivot_multi_index


성별과 흉통유형을 인덱스로 한 피벗테이블:


최대심박수
성별 흉통유형            
0  0     145.282051
   1     162.833333
   2     151.800000
   3     149.500000
1  0     138.759615
   1     162.187500
   2     158.173077
   3     157.315789

In [41]:
import pandas as pd

# 데이터 생성
data = {
    '성별': ['남성', '여성', '남성', '여성'],
    '흉통유형': [1, 2, 3, 1],
    '최대심박수': [150, 160, 155, 145]
}
df = pd.DataFrame(data)

# 피벗테이블 생성
pivot = df.pivot_table(index='성별', columns='흉통유형', values='최대심박수', aggfunc='mean')

# 결과 출력
print("피벗테이블 (결측값 포함):")
pivot

피벗테이블 (결측값 포함):


흉통유형,1,2,3
성별,,,
남성,150.0,NaN,155.0
여성,145.0,160.0,NaN


In [47]:
# 결측값을 0으로 대체
pivot_filled = df.pivot_table(index='성별', columns='흉통유형', values='최대심박수', aggfunc='mean', fill_value=0)
# 결과 출력
print("\n결측값을 0으로 대체한 피벗테이블:")
pivot_filled


결측값을 0으로 대체한 피벗테이블:


흉통유형,1,2,3
성별,,,
남성,150.0,0.0,155.0
여성,145.0,160.0,0.0


In [48]:
# 피벗테이블 생성 후 열 기준 평균값으로 대체
pivot_mean_filled = pivot.apply(lambda x: x.fillna(x.mean()), axis=0)
# 결과 출력
print("\n열 기준 평균값으로 대체한 피벗테이블:")
pivot_mean_filled


열 기준 평균값으로 대체한 피벗테이블:


흉통유형,1,2,3
성별,,,
남성,150.0,160.0,155.0
여성,145.0,160.0,155.0


In [49]:
# 피벗테이블 생성 후 결측값을 999로 대체
pivot_specific_value = pivot.fillna(999)
# 결과 출력
print("\n결측값을 특정 값(999)으로 대체한 피벗테이블:")
pivot_specific_value


결측값을 특정 값(999)으로 대체한 피벗테이블:


흉통유형,1,2,3
성별,,,
남성,150.0,999.0,155.0
여성,145.0,160.0,999.0


## <b> 인덱스와 컬럼 계층 변경

### 데이터준비

In [42]:
import pandas as pd

# 데이터 로드
file_path = "datasets/heart.csv"
df = pd.read_csv(file_path)

# 컬럼명 변경
df.columns = ['나이', '성별', '흉통유형', '안정시혈압', '콜레스테롤', '공복혈당', '심전도',
              '최대심박수', '운동유발협심증', 'ST우울증', '운동후ST경사', '주요혈관수',
              '결함유형', '심장질환여부']

# 데이터 확인
print("데이터프레임 상위 5개 행:")
print(df.head())

데이터프레임 상위 5개 행:
   나이  성별  흉통유형  안정시혈압  콜레스테롤  공복혈당  심전도  최대심박수  운동유발협심증  ST우울증  운동후ST경사  \
0  63   1     3    145    233     1    0    150        0    2.3        0   
1  37   1     2    130    250     0    1    187        0    3.5        0   
2  41   0     1    130    204     0    0    172        0    1.4        2   
3  56   1     1    120    236     0    1    178        0    0.8        2   
4  57   0     0    120    354     0    1    163        1    0.6        2   

   주요혈관수  결함유형  심장질환여부  
0      0     1       1  
1      0     2       1  
2      0     2       1  
3      0     2       1  
4      0     2       1  


### 그룹화 후 다중 인덱스 생성

In [51]:
# 성별과 흉통유형을 기준으로 그룹화하여 평균 계산
grouped = df.groupby(['성별', '흉통유형']).mean()

# 그룹화된 데이터 확인
print("\n성별과 흉통유형 기준 다중 인덱스:")
grouped


성별과 흉통유형 기준 다중 인덱스:


나이       안정시혈압       콜레스테롤      공복혈당       심전도       최대심박수  \
성별 흉통유형                                                                      
0  0     57.256410  138.589744  267.538462  0.128205  0.538462  145.282051   
   1     51.944444  128.055556  251.444444  0.111111  0.555556  162.833333   
   2     54.971429  127.885714  261.057143  0.114286  0.600000  151.800000   
   3     63.250000  147.500000  247.000000  0.250000  0.750000  149.500000   
1  0     55.105769  129.557692  243.605769  0.125000  0.451923  138.759615   
   1     51.031250  128.593750  241.031250  0.093750  0.656250  162.187500   
   2     52.538462  132.057692  231.134615  0.250000  0.596154  158.173077   
   3     54.315789  139.473684  235.052632  0.210526  0.315789  157.315789   

          운동유발협심증     ST우울증   운동후ST경사     주요혈관수      결함유형    심장질환여부  
성별 흉통유형                                                              
0  0     0.461538  1.356410  1.179487  0.820513  2.282051  0.461538  
   1     0.111111  0.461111  1.666667  0.555556  2.000000  0.888889  
   2     0.057143  0.474286  1.571429  0.257143  2.028571  0.971429  
   3     0.000000  1.575000  1.500000  0.500000  2.000000  1.000000  
1  0     0.596154  1.393269  1.288462  1.019231  2.509615  0.201923  
   1     0.062500  0.234375  1.687500  0.343750  2.218750  0.781250  
   2     0.173077  1.015385  1.461538  0.807692  2.326923  0.673077  
   3     0.210526  1.352632  1.210526  0.473684  2.315789  0.631579

### 인덱스 초기화

In [52]:
# 다중 인덱스 초기화
grouped_reset = grouped.reset_index()

print("\n다중 인덱스 초기화:")
grouped_reset


다중 인덱스 초기화:


,성별,흉통유형,나이,안정시혈압,콜레스테롤,공복혈당,심전도,최대심박수,운동유발협심증,ST우울증,운동후ST경사,주요혈관수,결함유형,심장질환여부
0,0,0,57.256410,138.589744,267.538462,0.128205,0.538462,145.282051,0.461538,1.356410,1.179487,0.820513,2.282051,0.461538
1,0,1,51.944444,128.055556,251.444444,0.111111,0.555556,162.833333,0.111111,0.461111,1.666667,0.555556,2.000000,0.888889
2,0,2,54.971429,127.885714,261.057143,0.114286,0.600000,151.800000,0.057143,0.474286,1.571429,0.257143,2.028571,0.971429
3,0,3,63.250000,147.500000,247.000000,0.250000,0.750000,149.500000,0.000000,1.575000,1.500000,0.500000,2.000000,1.000000
4,1,0,55.105769,129.557692,243.605769,0.125000,0.451923,138.759615,0.596154,1.393269,1.288462,1.019231,2.509615,0.201923
5,1,1,51.031250,128.593750,241.031250,0.093750,0.656250,162.187500,0.062500,0.234375,1.687500,0.343750,2.218750,0.781250
6,1,2,52.538462,132.057692,231.134615,0.250000,0.596154,158.173077,0.173077,1.015385,1.461538,0.807692,2.326923,0.673077
7,1,3,54.315789,139.473684,235.052632,0.210526,0.315789,157.315789,0.210526,1.352632,1.210526,0.473684,2.315789,0.631579


### pandas 라이브러리 데이터프레임 출력의 스타일링 기능

In [43]:
df

,나이,성별,흉통유형,안정시혈압,콜레스테롤,공복혈당,심전도,최대심박수,운동유발협심증,ST우울증,운동후ST경사,주요혈관수,결함유형,심장질환여부
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [45]:
pd.set_option('display.max_columns', None) # 모든 열 출력 
pd.set_option('display.width', 1200) # 데이터프레임 너비 조정
pd.set_option('display.float_format', '{:.2f}'.format) # 소수점 형식 설정

### 다중 컬럼 피벗테이블 생성

In [54]:
# 다중 컬럼 피벗테이블 생성
pivot = df.pivot_table(index='성별', columns=['흉통유형', '운동후ST경사'], values='최대심박수', aggfunc='mean')

print("\n다중 컬럼 피벗테이블:")
pivot


다중 컬럼 피벗테이블:


흉통유형         0                    1                    2                    3              
운동후ST경사      0      1      2      0      1      2      0      1      2      0      1      2
성별                                                                                         
0       146.00 139.00 161.40    NaN 160.50 164.00 160.00 149.15 153.05 114.00    NaN 161.33
1       126.75 128.43 157.05 168.50 139.83 167.25 171.25 147.25 164.11 147.50 159.27 157.00

### 컬럼 계층 초기화

In [55]:
# 컬럼 계층 초기화
pivot_reset = pivot.reset_index()
pivot_reset.columns = ['_'.join(map(str, col)).strip() for col in pivot_reset.columns]

print("\n컬럼 계층 초기화:")
pivot_reset


컬럼 계층 초기화:


,성별_,0_0,0_1,0_2,1_0,1_1,1_2,2_0,2_1,2_2,3_0,3_1,3_2
0,0,146.00,139.00,161.40,NaN,160.50,164.00,160.00,149.15,153.05,114.00,NaN,161.33
1,1,126.75,128.43,157.05,168.50,139.83,167.25,171.25,147.25,164.11,147.50,159.27,157.00


### 컬럼 계층 전환

In [56]:
# 다중 컬럼을 다중 인덱스로 전환
pivot_stack = pivot.stack()

print("\n다중 컬럼을 다중 인덱스로 전환:")
pivot_stack


다중 컬럼을 다중 인덱스로 전환:


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35152\3964089219.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  pivot_stack = pivot.stack()


흉통유형            0      1      2      3
성별 운동후ST경사                            
0  0       146.00    NaN 160.00 114.00
   1       139.00 160.50 149.15    NaN
   2       161.40 164.00 153.05 161.33
1  0       126.75 168.50 171.25 147.50
   1       128.43 139.83 147.25 159.27
   2       157.05 167.25 164.11 157.00

### 경고 메시지 차단

In [57]:
import warnings # 경고 메시지 차단 
warnings.filterwarnings('ignore')

# <b> 3.데이터 병합과 추가

## <B> (1) 데이터 병합과 추가와 중요성

## <B> (2) 데이터 병합()

### 데이터준비

In [46]:
import pandas as pd

# 첫 번째 데이터프레임 생성
data1 = {
    '제품ID': [1, 2, 3],
    '제품명': ['노트북', '모니터', '키보드']
}
df1 = pd.DataFrame(data1)

# 두 번째 데이터프레임 생성
data2 = {
    '제품ID': [1, 2, 4],
    '가격': [1000, 200, 50]
}
df2 = pd.DataFrame(data2)


In [49]:
df1

,제품ID,제품명
0,1,노트북
1,2,모니터
2,3,키보드


In [50]:
df2

,제품ID,가격
0,1,1000
1,2,200
2,4,50


### 내부 조인(Inner Join)

In [51]:
# 내부 조인
inner_join = pd.merge(df1, df2, on='제품ID', how='inner')

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)
print("\n내부 조인 결과:")
print(inner_join)

첫 번째 데이터프레임:
   제품ID  제품명
0     1  노트북
1     2  모니터
2     3  키보드

두 번째 데이터프레임:
   제품ID    가격
0     1  1000
1     2   200
2     4    50

내부 조인 결과:
   제품ID  제품명    가격
0     1  노트북  1000
1     2  모니터   200


### 외부 조인(Outer Join)

In [52]:
# 외부 조인
outer_join = pd.merge(df1, df2, on='제품ID', how='outer')

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)
print("\n외부 조인 결과:")
print(outer_join)

첫 번째 데이터프레임:
   제품ID  제품명
0     1  노트북
1     2  모니터
2     3  키보드

두 번째 데이터프레임:
   제품ID    가격
0     1  1000
1     2   200
2     4    50

외부 조인 결과:
   제품ID  제품명      가격
0     1  노트북 1000.00
1     2  모니터  200.00
2     3  키보드     NaN
3     4  NaN   50.00


### 왼쪽 조인(left join)

In [53]:
# 왼쪽 조인
left_join = pd.merge(df1, df2, on='제품ID', how='left')

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)
print("\n왼쪽 조인 결과:")
print(left_join)

첫 번째 데이터프레임:
   제품ID  제품명
0     1  노트북
1     2  모니터
2     3  키보드

두 번째 데이터프레임:
   제품ID    가격
0     1  1000
1     2   200
2     4    50

왼쪽 조인 결과:
   제품ID  제품명      가격
0     1  노트북 1000.00
1     2  모니터  200.00
2     3  키보드     NaN


### 오른쪽 조인(Right join)

In [54]:
# 오른쪽 조인
right_join = pd.merge(df1, df2, on='제품ID', how='right')

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)
print("\n오른쪽 조인 결과:")
print(right_join)

첫 번째 데이터프레임:
   제품ID  제품명
0     1  노트북
1     2  모니터
2     3  키보드

두 번째 데이터프레임:
   제품ID    가격
0     1  1000
1     2   200
2     4    50

오른쪽 조인 결과:
   제품ID  제품명    가격
0     1  노트북  1000
1     2  모니터   200
2     4  NaN    50


### 교차 조인(Cross Join)

In [63]:
# 교차 조인 
cross_join = pd.merge(df1, df2, how='cross') 

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)
print("\n교차 조인 결과:") 
print(cross_join)

첫 번째 데이터프레임:
   제품ID  제품명
0     1  노트북
1     2  모니터
2     3  키보드

두 번째 데이터프레임:
   제품ID    가격
0     1  1000
1     2   200
2     4    50

교차 조인 결과:
   제품ID_x  제품명  제품ID_y    가격
0       1  노트북       1  1000
1       1  노트북       2   200
2       1  노트북       4    50
3       2  모니터       1  1000
4       2  모니터       2   200
5       2  모니터       4    50
6       3  키보드       1  1000
7       3  키보드       2   200
8       3  키보드       4    50


## <B> (3) 데이터 추가(Union)

### 데이터준비

In [55]:
# 첫 번째 데이터프레임 생성
data1 = {
    '이름': ['홍길동', '이순신'],
    '나이': [30, 40]
}
df1 = pd.DataFrame(data1)

# 두 번째 데이터프레임 생성
data2 = {
    '이름': ['강감찬', '유관순'],
    '나이': [50, 20]
}
df2 = pd.DataFrame(data2)

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)

첫 번째 데이터프레임:
    이름  나이
0  홍길동  30
1  이순신  40

두 번째 데이터프레임:
    이름  나이
0  강감찬  50
1  유관순  20


### 데이터프레임 결합(Union) : ignore_index = True

In [59]:
# 데이터프레임 세로로 결합
union_df = pd.concat([df1, df2], ignore_index=True)

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)
print("\n유니온 결과:")
print(union_df)

첫 번째 데이터프레임:
    이름  나이
0  홍길동  30
1  이순신  40

두 번째 데이터프레임:
    이름  나이
0  강감찬  50
1  유관순  20

유니온 결과:
    이름  나이
0  홍길동  30
1  이순신  40
2  강감찬  50
3  유관순  20


###  데이터프레임 결합(Union) : ignore_index = False

In [66]:
# 인덱스를 유지하면서 데이터프레임 결합
union_with_index = pd.concat([df1, df2], ignore_index=False)

# 데이터프레임 출력
print("첫 번째 데이터프레임:")
print(df1)
print("\n두 번째 데이터프레임:")
print(df2)
print("\n인덱스를 유지한 유니온 결과:")
print(union_with_index)

첫 번째 데이터프레임:
    이름  나이
0  홍길동  30
1  이순신  40

두 번째 데이터프레임:
    이름  나이
0  강감찬  50
1  유관순  20

인덱스를 유지한 유니온 결과:
    이름  나이
0  홍길동  30
1  이순신  40
0  강감찬  50
1  유관순  20


### 데이터프레임 열이 다른 경우 결합(Union) 

In [62]:
# 열이 다른 데이터프레임 생성
data3 = {
    '이름': ['정약용', '김유신'],
    '주소': ['서울', '경주']
}
df3 = pd.DataFrame(data3)


In [63]:
print(df1)
print(df2)
print(df3)

    이름  나이
0  홍길동  30
1  이순신  40
    이름  나이
0  강감찬  50
1  유관순  20
    이름  주소
0  정약용  서울
1  김유신  경주


In [65]:

# 데이터프레임 결합
union_with_different_columns = pd.concat([df1 , df2,  df3], ignore_index=True)

print("\n열이 다른 데이터프레임 결합 결과:")
print(union_with_different_columns)


열이 다른 데이터프레임 결합 결과:
    이름    나이   주소
0  홍길동 30.00  NaN
1  이순신 40.00  NaN
2  강감찬 50.00  NaN
3  유관순 20.00  NaN
4  정약용   NaN   서울
5  김유신   NaN   경주


## <B> (4) 데이터 파생(변수생성)

In [73]:
import pandas as pd
import numpy as np



# 연령대 변수 생성
df['연령대'] = pd.cut(df['나이'], bins=[0, 30, 40, 50, 100], 
		labels=['20대', '30대', '40대', '50대 이상'])

# 누적 월급 계산
df['누적월급'] = df['월급'].cumsum()

# 평균 대비 차이 계산
df['월급_평균차'] = df['월급'] - df['월급'].mean()

# 로그 변환 (정규화)
df['월급_로그'] = np.log1p(df['월급'])

print(df)


   이름  나이   월급     연령대  누적월급  월급_평균차  월급_로그
0  철수  23  250     20대   250 -237.50   5.53
1  영희  35  400     30대   650  -87.50   5.99
2  민수  45  600     40대  1250  112.50   6.40
3  지수  52  700  50대 이상  1950  212.50   6.55


In [67]:
# 샘플 데이터 생성
df = pd.DataFrame({
    '이름': ['철수', '영희', '민수', '지수'],
    '나이': [23, 35, 45, 52],
    '월급': [250, 400, 600, 700]
})

In [68]:
df

,이름,나이,월급
0,철수,23,250
1,영희,35,400
2,민수,45,600
3,지수,52,700


In [69]:
# 누적 월급 계산
df['누적월급'] = df['월급'].cumsum()
df

,이름,나이,월급,누적월급
0,철수,23,250,250
1,영희,35,400,650
2,민수,45,600,1250
3,지수,52,700,1950


In [71]:
# 평균 대비 차이 계산
df['월급_평균차'] = df['월급'] - df['월급'].mean()
df

,이름,나이,월급,누적월급,월급_평균차
0,철수,23,250,250,-237.50
1,영희,35,400,650,-87.50
2,민수,45,600,1250,112.50
3,지수,52,700,1950,212.50


In [74]:
df = pd.DataFrame({
    '이름': ['철수', '영희', '민수', '지수'],
    '성별': ['남', '여', '남', '여'],
    '주소': ['서울 강남구', '부산 해운대구', '서울 종로구', '대전 서구']
})

# 성별을 숫자로 변환
df['성별_코드'] = df['성별'].map({'남': 0, '여': 1})

# 주소에서 도시와 구 추출
df['도시'] = df['주소'].str.split().str[0]
df['구'] = df['주소'].str.split().str[1]

print(df)


   이름 성별       주소  성별_코드  도시     구
0  철수  남   서울 강남구      0  서울   강남구
1  영희  여  부산 해운대구      1  부산  해운대구
2  민수  남   서울 종로구      0  서울   종로구
3  지수  여    대전 서구      1  대전    서구


In [75]:
df = pd.DataFrame({
    '이름': ['철수', '영희', '민수', '지수'],
    '구매일': pd.to_datetime(['2023-01-15', '2023-05-20', '2023-08-10', '2023-12-25'])
})

# 날짜 정보 파생
df['연도'] = df['구매일'].dt.year
df['월'] = df['구매일'].dt.month
df['요일'] = df['구매일'].dt.day_name()
df['주말여부'] = df['요일'].isin(['Saturday', 'Sunday']).astype(int)
df['분기'] = df['구매일'].dt.quarter

print(df)


   이름        구매일    연도   월        요일  주말여부  분기
0  철수 2023-01-15  2023   1    Sunday     1   1
1  영희 2023-05-20  2023   5  Saturday     1   2
2  민수 2023-08-10  2023   8  Thursday     0   3
3  지수 2023-12-25  2023  12    Monday     0   4


In [77]:
import pandas as pd
import numpy as np

# 샘플 데이터 생성
df = pd.DataFrame({
    '이름': ['철수', '영희', '민수', '지수'],
    '총매출': [10000, 20000, 30000, 40000],
    '고객수': [100, 150, 200, 250],
    '광고비': [500, 700, 1000, 1500],
    '리뷰점수': [4.2, 4.5, 3.8, 4.0],
    '리뷰개수': [30, 50, 20, 40]
})

df['1인당 소비금액'] = df['총매출'] / df['고객수']  # 총매출 대비 고객 1명당 평균 소비금액
df['광고효과'] = df['총매출'] / df['광고비']  # 광고비 대비 매출 효과
df['매출 증가율'] = df['총매출'].pct_change().fillna(0)  # 전월 대비 매출 증가율
df['리뷰 점수 대비 차이'] = df['리뷰점수'] - df['리뷰점수'].mean()  # 평균 대비 리뷰 점수 차이
df['리뷰 품질 점수'] = df['리뷰점수'] * df['리뷰개수']  # 리뷰 점수 × 리뷰 개수 (리뷰의 신뢰도 반영)
df


,이름,총매출,고객수,광고비,리뷰점수,리뷰개수,1인당 소비금액,광고효과,매출 증가율,리뷰 점수 대비 차이,리뷰 품질 점수
0,철수,10000,100,500,4.20,30,100.00,20.00,0.00,0.08,126.00
1,영희,20000,150,700,4.50,50,133.33,28.57,1.00,0.38,225.00
2,민수,30000,200,1000,3.80,20,150.00,30.00,0.50,-0.33,76.00
3,지수,40000,250,1500,4.00,40,160.00,26.67,0.33,-0.12,160.00
